In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def fetch_tracking_details(driver, awbid):
    driver.get(f"https://www.delhivery.com/track-v2/package/{awbid}")

    try:
        # Wait for the desired containers
        containers = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'flex items-start w-full')]")))

        for container in containers:
            # Check for specific nested class
            if container.find_elements(By.XPATH, ".//div[contains(@class, 'bg-success w-[40px] h-[40px] flex justify-center items-center rounded-full')]"):
                # Print the text of the desired class
                status_text = container.find_element(By.XPATH, ".//div[contains(@class, 'text-[12px] font-semibold')]").text
                print(status_text)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()

if __name__ == "__main__":
    driver = initialize_driver()
    fetch_tracking_details(driver, "11311514505841")


IN-TRANSIT


In [13]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB 435.7 kB/s eta 0:00:37
   ---------------------------------------- 0.1/16.1 MB 1.2 MB/s eta 0:00:14
   - -------------------------------------- 0.7/16.1 MB 4.7 MB/s eta 0:00:04
   --- ------------------------------------ 1.4/16.1 MB 6.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.9/16.1 MB 7.5 MB/s eta 0:00:02
   ------ --------------------------------- 2.8/16.1 MB 9.4 MB/s eta 0:00:02
   -------- ------------------------------- 3.6/16.1 MB 10.3 MB/s eta 0:00:02
   ---------- ----------------------------- 4.2/16.1 MB 11.1 MB/s eta 0:00:02
   ---------- ----------------------------- 4.3/16.1 MB 10.7 MB/s eta 0:00:02
   ----------- ---------------------------- 4.4/16.1 MB 10.1 MB/s eta 0:00:02
   ----------- ---------------------------- 4.5/16.1 MB 8.9 MB/s eta 0:00:02
   ----


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import sys
import mysql.connector
from mysql.connector import Error
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def initialize_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    return driver

def fetch_tracking_details(driver, awbid):
    driver.get(f"https://www.delhivery.com/track-v2/package/{awbid}")

    try:
        containers = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'flex items-start w-full')]")))

        for container in containers:
            if container.find_elements(By.XPATH, ".//div[contains(@class, 'bg-success w-[40px] h-[40px] flex justify-center items-center rounded-full')]"):
                status_text = container.find_element(By.XPATH, ".//div[contains(@class, 'text-[12px] font-semibold')]").text
                return status_text
    except Exception as e:
        return f"An error occurred: {e}"
    finally:
        driver.quit()

def connect_to_database(host, user, password, database):
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        if connection.is_connected():
            return connection
    except Error as e:
        return f"Error while connecting to MySQL: {e}"

if __name__ == "__main__":
    awbid = sys.argv[1]
    db_host = sys.argv[2]
    db_user = sys.argv[3]
    db_password = sys.argv[4]
    db_name = sys.argv[5]
    
    driver = initialize_driver()
    status = fetch_tracking_details(driver, awbid)

    connection = connect_to_database(db_host, db_user, db_password, db_name)
    if isinstance(connection, str):
        print(connection)  # Error message
    else:
        cursor = connection.cursor()
        cursor.execute("INSERT INTO tracking_info (awb_id, status) VALUES (%s, %s)", (awbid, status))
        connection.commit()
        print(f"Tracking details for {awbid} saved to database.")
        connection.close()


IndexError: list index out of range

In [1]:
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import logging
import configparser

# Set up logging
logging.basicConfig(filename='tracking.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def initialize_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run browser in headless mode
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    logging.debug('Web driver initialized')
    return driver

def fetch_tracking_details(driver, awbid):
    logging.debug(f'Fetching tracking details for AWB ID: {awbid}')
    driver.get(f"https://www.delhivery.com/track-v2/package/{awbid}")

    try:
        containers = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'flex items-start w-full')]")))
        logging.debug(f'Containers found: {len(containers)}')
        for container in containers:
            if container.find_elements(By.XPATH, ".//div[contains(@class, 'bg-success w-[40px] h-[40px] flex justify-center items-center rounded-full')]"):
                status_text = container.find_element(By.XPATH, ".//div[contains(@class, 'text-[12px] font-semibold')]").text
                return status_text
    except Exception as e:
        logging.error(f'Error fetching tracking details: {e}')
        return f"Error: {e}"
    finally:
        driver.quit()
        logging.debug('Web driver quit')

    return "No status"

def update_tracking_status():
    config = configparser.ConfigParser()
    config.read('config.ini')  # Read the config file

    conn = None
    cursor = None
    driver = None
    try:
        # Connect to the database using values from config.ini
        conn = mysql.connector.connect(
            host=config['database']['host'],
            user=config['database']['user'],
            password=config['database']['password'],
            database=config['database']['name']
        )
        cursor = conn.cursor()
        logging.debug('Database connection established')

        # Fetch all tracking IDs
        cursor.execute("SELECT id, tracking_id FROM shipment_tracking_info")
        rows = cursor.fetchall()
        logging.debug(f'Fetched {len(rows)} tracking IDs from the database')

        # Initialize the web driver
        driver = initialize_driver()

        for row in rows:
            tracking_id = row[1]
            status = fetch_tracking_details(driver, tracking_id)

            # Update the status in the database
            cursor.execute("UPDATE shipment_tracking_info SET status=%s WHERE id=%s", (status, row[0]))
            conn.commit()
            logging.debug(f'Status updated for ID: {row[0]}')

    except mysql.connector.Error as err:
        logging.error(f'Database error: {err}')
    except Exception as e:
        logging.error(f'Error in update_tracking_status: {e}')
    finally:
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()
        if driver is not None:
            driver.quit()
        logging.debug('Database connection closed and web driver quit')

if __name__ == "__main__":
    update_tracking_status()


In [32]:
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import logging
import configparser
import time

# Set up logging
logging.basicConfig(filename='tracking.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def initialize_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run browser in headless mode
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    logging.debug('Web driver initialized')
    return driver

def fetch_tracking_details(awbid):
    logging.debug(f'Fetching tracking details for AWB ID: {awbid}')
    driver = initialize_driver()
    driver.get(f"https://www.delhivery.com/track-v2/package/{awbid}")

    try:
        containers = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'flex items-start w-full')]")))
        logging.debug(f'Containers found: {len(containers)}')
        for container in containers:
            if container.find_elements(By.XPATH, ".//div[contains(@class, 'bg-success w-[40px] h-[40px] flex justify-center items-center rounded-full')]"):
                status_text = container.find_element(By.XPATH, ".//div[contains(@class, 'text-[12px] font-semibold')]").text
                logging.debug(f'Status found: {status_text}')
                return status_text
    except Exception as e:
        logging.error(f'Error fetching tracking details: {e}')
        return f"Error: {e}"
    finally:
        driver.quit()
        logging.debug('Web driver quit')

    return "No status"

def update_tracking_status():
    config = configparser.ConfigParser()
    config.read('config.ini')  # Read the config file

    conn = None
    cursor = None
    try:
        # Connect to the database using values from config.ini
        conn = mysql.connector.connect(
            host=config['database']['host'],
            user=config['database']['user'],
            password=config['database']['password'],
            database=config['database']['name']
        )
        cursor = conn.cursor()
        logging.debug('Database connection established')

        # Fetch all tracking IDs
        cursor.execute("SELECT id, tracking_id FROM shipment_tracking_info")
        rows = cursor.fetchall()
        logging.debug(f'Fetched {len(rows)} tracking IDs from the database')

        for row in rows:
            tracking_id = row[1]
            status = fetch_tracking_details(tracking_id)

            # Update the status in the database
            cursor.execute("UPDATE shipment_tracking_info SET status=%s WHERE id=%s", (status, row[0]))
            conn.commit()
            logging.debug(f'Status updated for ID: {row[0]}')
            time.sleep(2)  # Delay to ensure stability

    except mysql.connector.Error as err:
        logging.error(f'Database error: {err}')
    except Exception as e:
        logging.error(f'Error in update_tracking_status: {e}')
    finally:
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()
        logging.debug('Database connection closed')

if __name__ == "__main__":
    update_tracking_status()


In [4]:
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import logging
import configparser
import time

# Set up logging
logging.basicConfig(filename='tracking.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def initialize_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run browser in headless mode
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    logging.debug('Web driver initialized')
    return driver

def fetch_tracking_details(awbid):
    logging.debug(f'Fetching tracking details for AWB ID: {awbid}')
    driver = initialize_driver()
    driver.get(f"https://www.delhivery.com/track-v2/package/{awbid}")

    try:
        # Check for the "Could not find the order with us" message
        error_message_div = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'flex mx-6 w-[500px] flex-col items-center p-4 bg-white rounded-md')]"))
        )
        logging.debug('Error message found: Could not find the order with us')
        return "Check Tracking ID"
    except:
        logging.debug('Error message not found: Proceeding to next check')
        pass

    try:
        # Check for the "Oops! 404 Error" message
        not_found_message_div = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'not-found-text-wrap')]"))
        )
        logging.debug('Error message found: You’re on the wrong route')
        return "Wrong Tracking ID"
    except:
        logging.debug('404 Error message not found: Proceeding to status check')
        pass
    
    try:
        containers = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'flex items-start w-full')]")))
        logging.debug(f'Containers found: {len(containers)}')
        for container in containers:
            if container.find_elements(By.XPATH, ".//div[contains(@class, 'bg-success w-[40px] h-[40px] flex justify-center items-center rounded-full')]"):
                status_text = container.find_element(By.XPATH, ".//div[contains(@class, 'text-[12px] font-semibold')]").text
                logging.debug(f'Status found: {status_text}')
                return status_text
    except Exception as e:
        logging.error(f'Error fetching tracking details: {e}')
        return f"Error: {e}"
    finally:
        driver.quit()
        logging.debug('Web driver quit')

    return "No status"

def update_tracking_status():
    config = configparser.ConfigParser()
    config.read('config.ini')  # Read the config file

    conn = None
    cursor = None
    try:
        # Connect to the database using values from config.ini
        conn = mysql.connector.connect(
            host=config['database']['host'],
            user=config['database']['user'],
            password=config['database']['password'],
            database=config['database']['name']
        )
        cursor = conn.cursor()
        logging.debug('Database connection established')

        # Fetch all tracking IDs
        cursor.execute("SELECT id, tracking_id FROM shipment_tracking_info")
        rows = cursor.fetchall()
        logging.debug(f'Fetched {len(rows)} tracking IDs from the database')

        for row in rows:
            tracking_id = row[1]
            status = fetch_tracking_details(tracking_id)

            # Update the status in the database
            cursor.execute("UPDATE shipment_tracking_info SET status=%s WHERE id=%s", (status, row[0]))
            conn.commit()
            logging.debug(f'Status updated for ID: {row[0]}')
            time.sleep(2)  # Delay to ensure stability

    except mysql.connector.Error as err:
        logging.error(f'Database error: {err}')
    except Exception as e:
        logging.error(f'Error in update_tracking_status: {e}')
    finally:
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()
        logging.debug('Database connection closed')

if __name__ == "__main__":
    update_tracking_status()

In [2]:
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import logging
import configparser
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Set up logging
logging.basicConfig(filename='tracking.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def initialize_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run browser in headless mode
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    logging.debug('Web driver initialized')
    return driver

def fetch_tracking_details(awbid):
    logging.debug(f'Fetching tracking details for AWB ID: {awbid}')
    driver = initialize_driver()
    driver.get(f"https://www.delhivery.com/track-v2/package/{awbid}")

    try:
        # Check for the "Could not find the order with us" message
        error_message_div = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'flex mx-6 w-[500px] flex-col items-center p-4 bg-white rounded-md')]"))
        )
        logging.debug('Error message found: Could not find the order with us')
        return "Check Tracking ID"
    except:
        logging.debug('Error message not found: Proceeding to next check')
        pass

    try:
        # Check for the "Oops! 404 Error" message
        not_found_message_div = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'not-found-text-wrap')]"))
        )
        logging.debug('Error message found: You’re on the wrong route')
        return "Wrong Tracking ID"
    except:
        logging.debug('404 Error message not found: Proceeding to status check')
        pass
    
    try:
        containers = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'flex items-start w-full')]")))
        logging.debug(f'Containers found: {len(containers)}')
        for container in containers:
            if container.find_elements(By.XPATH, ".//div[contains(@class, 'bg-success w-[40px] h-[40px] flex justify-center items-center rounded-full')]"):
                status_text = container.find_element(By.XPATH, ".//div[contains(@class, 'text-[12px] font-semibold')]").text
                logging.debug(f'Status found: {status_text}')
                return status_text
    except Exception as e:
        logging.error(f'Error fetching tracking details: {e}')
        return f"Error: {e}"
    finally:
        driver.quit()
        logging.debug('Web driver quit')

    return "No status"

def send_email(email, order_id, tracking_id, status, purchase_date, seller_name):
    config = configparser.ConfigParser()
    config.read('config.ini')  # Read the config file

    from_email = config['email']['from_email']
    from_password = config['email']['password']
    smtp_server = config['email']['smtp_server']
    smtp_port = config['email']['smtp_port']

    subject = f"Order Update for {order_id}"
    body = f"""
    Hi {seller_name},

    Your order with ID {order_id} purchased on {purchase_date} is currently at the following status: {status}.

    Tracking ID: {tracking_id}

    Thank you,
    Your Company Name
    """

    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP_SSL(smtp_server, smtp_port)
        server.login(from_email, from_password)
        server.sendmail(from_email, email, msg.as_string())
        server.quit()
        logging.debug(f"Email sent successfully to {email}")
    except Exception as e:
        logging.error(f"Failed to send email: {e}")

def update_tracking_status():
    config = configparser.ConfigParser()
    config.read('config.ini')  # Read the config file

    conn = None
    cursor = None
    try:
        # Connect to the database using values from config.ini
        conn = mysql.connector.connect(
            host=config['database']['host'],
            user=config['database']['user'],
            password=config['database']['password'],
            database=config['database']['name']
        )
        cursor = conn.cursor()
        logging.debug('Database connection established')

        # Fetch all tracking IDs
        cursor.execute("SELECT id, tracking_id, email, order_id, purchase_date, seller_name FROM shipment_tracking_info")
        rows = cursor.fetchall()
        logging.debug(f'Fetched {len(rows)} tracking IDs from the database')

        for row in rows:
            tracking_id = row[1]
            email = row[2]
            order_id = row[3]
            purchase_date = row[4]
            seller_name = row[5]
            status = fetch_tracking_details(tracking_id)

            # Update the status in the database
            cursor.execute("UPDATE shipment_tracking_info SET status=%s WHERE id=%s", (status, row[0]))
            conn.commit()
            logging.debug(f'Status updated for ID: {row[0]}')

            # Send email notification
            send_email(email, order_id, tracking_id, status, purchase_date, seller_name)
            time.sleep(2)  # Delay to ensure stability

    except mysql.connector.Error as err:
        logging.error(f'Database error: {err}')
    except Exception as e:
        logging.error(f'Error in update_tracking_status: {e}')
    finally:
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()
        logging.debug('Database connection closed')

if __name__ == "__main__":
    update_tracking_status()

In [4]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Uncomment this if you don't want the browser window to be visible
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.safexpress.com/")

# Close the pop-up
try:
    close_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn-close')))
    ActionChains(driver).move_to_element(close_button).click().perform()
except:
    pass

# Select Waybill option
waybill_radio = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'waybill_number')))
ActionChains(driver).move_to_element(waybill_radio).click().perform()

# Enter the Waybill Number
waybill_number = '100018408252'  # Replace with your actual Waybill Number
tracking_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[formcontrolname="trackingValue"]')))
tracking_input.send_keys(waybill_number)

# Click the Track button
track_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.track-button')))
ActionChains(driver).move_to_element(track_button).click().perform()

# Wait for the page to load tracking details
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[style*="border: 1px solid #006431"]')))

# Extract the status elements
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find all div elements that match the tracking steps
status_blocks = soup.find_all('div', style=lambda x: x and 'border: 1px solid #006431' in x)

# Variable to store the last status
last_status = ""

# Loop through each status block and store the last status text
for block in status_blocks:
    status_text = block.find('span', style=lambda x: x and 'font-weight: 550' and 'color: #006431' in x)
    if status_text:
        last_status = status_text.text.strip()  # Store the last status

# Print the last status after the loop finishes
print("Last Status:", last_status)

# Close the driver after scraping
driver.quit()


Last Status: DELIVERED


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Uncomment this if you don't want the browser window to be visible
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def start_driver():
    print("Starting WebDriver...")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver = start_driver()
    driver.get("https://www.safexpress.com/")

    # Close the pop-up
    try:
        close_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn-close')))
        ActionChains(driver).move_to_element(close_button).click().perform()
        print("Pop-up closed.")
    except Exception as e:
        print("No pop-up found or failed to close:", e)

    # Select Waybill option
    try:
        waybill_radio = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, 'waybill_number')))
        ActionChains(driver).move_to_element(waybill_radio).click().perform()
        print("Waybill option selected.")
    except Exception as e:
        print("Failed to select Waybill option:", e)

    # Enter the Waybill Number
    waybill_number = '10001868359'  # Replace with your actual Waybill Number
    try:
        tracking_input = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[formcontrolname="trackingValue"]')))
        tracking_input.send_keys(waybill_number)
        print(f"Waybill Number entered: {waybill_number}")
    except Exception as e:
        print("Failed to enter Waybill Number:", e)

    # Click the Track button
    try:
        track_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.track-button')))
        driver.execute_script("arguments[0].click();", track_button)  # Use JavaScript to click
        print("Track button clicked.")
    except Exception as e:
        print("Failed to click Track button:", e)

    # Wait for the presence of the tracking result container
    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.container.trackResult.for-desktop.ng-star-inserted')))
        print("Tracking result found, proceeding with status extraction.")
        
        # Extract the status elements using Selenium
        print("Extracting status elements...")
        status_blocks = driver.find_elements(By.CSS_SELECTOR, 'div[style*="border: 1px solid #006431"]')

        # Variable to store the last status
        last_status = ""

        # Loop through each status block and store the last status text
        for block in status_blocks:
            try:
                status_text = block.find_element(By.CSS_SELECTOR, 'span[style*="font-weight: 550; color: #006431;"]').text.strip()
                last_status = status_text  # Store the last status
                print("Status found:", last_status)
            except Exception as e:
                print("Status extraction failed for a block:", e)

        # Print the last status after the loop finishes
        print("Last Status:", last_status)
        
    except:
        print("Error: Unable to load data of Waybill Number. Exiting script.")
        driver.quit()
        exit()

except Exception as e:
    print("Error encountered:", e)
finally:
    if 'driver' in locals():
        driver.quit()
    print("WebDriver closed.")


Starting WebDriver...
Pop-up closed.
Waybill option selected.
Waybill Number entered: 10001868359
Track button clicked.
Error: Unable to load data of Waybill Number. Exiting script.
WebDriver closed.


: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Uncomment this if you don't want the browser window to be visible
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def start_driver():
    print("Starting WebDriver...")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver = start_driver()
    driver.get("https://www.safexpress.com/")

    # Close the pop-up
    try:
        close_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn-close')))
        ActionChains(driver).move_to_element(close_button).click().perform()
        print("Pop-up closed.")
    except Exception as e:
        print("No pop-up found or failed to close:", e)

    # Select Waybill option
    try:
        waybill_radio = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, 'waybill_number')))
        ActionChains(driver).move_to_element(waybill_radio).click().perform()
        print("Waybill option selected.")
    except Exception as e:
        print("Failed to select Waybill option:", e)

    # Enter the Waybill Number
    waybill_number = '10001868359'  # Replace with your actual Waybill Number
    try:
        tracking_input = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[formcontrolname="trackingValue"]')))
        tracking_input.send_keys(waybill_number)
        print(f"Waybill Number entered: {waybill_number}")
    except Exception as e:
        print("Failed to enter Waybill Number:", e)

    # Click the Track button
    try:
        track_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.track-button')))
        driver.execute_script("arguments[0].click();", track_button)  # Use JavaScript to click
        print("Track button clicked.")
    except Exception as e:
        print("Failed to click Track button:", e)

    # Wait for the presence of the tracking result container
    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.container.trackResult.for-desktop.ng-star-inserted')))
        print("Tracking result found, proceeding with status extraction.")
        
        # Extract the status elements using Selenium
        print("Extracting status elements...")
        status_blocks = driver.find_elements(By.CSS_SELECTOR, 'div[style*="border: 1px solid #006431"]')

        # Variable to store the last status
        last_status = ""

        # Loop through each status block and store the last status text
        for block in status_blocks:
            try:
                status_text = block.find_element(By.CSS_SELECTOR, 'span[style*="font-weight: 550; color: #006431;"]').text.strip()
                last_status = status_text  # Store the last status
                print("Status found:", last_status)
            except Exception as e:
                print("Status extraction failed for a block:", e)

        # Print the last status after the loop finishes
        print("Last Status:", last_status)
        
    except:
        print("Error: Unable to load data of Waybill Number. Exiting script.")
        driver.quit()
        exit()

except Exception as e:
    print("Error encountered:", e)
finally:
    if 'driver' in locals():
        driver.quit()
    print("WebDriver closed.")


Starting WebDriver...
Pop-up closed.
Waybill option selected.
Waybill Number entered: 10001868359
Track button clicked.
Error: Unable to load data of Waybill Number. Exiting script.
WebDriver closed.


: 